In [8]:
!pip install --quiet langchain openai langgraph faiss-cpu tiktoken langchain-community


In [9]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [10]:
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.3)

# Tool 1: Job Description Generator
def generate_job_description(role: str) -> str:
    prompt = (
        f"Create a job description for a {role} at SquareShift. "
        "Highlight responsibilities in digital transformation, data engineering, "
        "cybersecurity, and cloud adoption. Emphasize our presence in the USA, Singapore, and India, "
        "and our commitment to innovation and client success."
    )
    return llm.predict(prompt)

jd_tool = Tool(
    name="JobDescriptionGenerator",
    func=generate_job_description,
    description="Generates a job description for a given role at SquareShift"
)

# Tool 2: Hiring Plan Generator
def generate_hiring_plan(role: str) -> str:
    return (
        f"The hiring plan for a {role} at SquareShift includes: "
        "1) Sourcing candidates through tech communities and platforms; "
        "2) Screening resumes for alignment with our expertise in AI, ML, and cloud services; "
        "3) Conducting technical interviews focusing on problem-solving and innovation; "
        "4) Final interviews to assess cultural fit and commitment to our mission; "
        "5) Rolling out offers with competitive compensation and growth opportunities."
    )

plan_tool = Tool(
    name="HiringPlanTool",
    func=generate_hiring_plan,
    description="Provides a 5-step hiring plan for a given role at SquareShift"
)

# Tool 3: Cost Estimator
def estimate_cost(role: str) -> str:
    base_cost = {
        "Senior Software Engineer": 15000,
        "DevOps Engineer": 14000,
        "Data Analyst": 13000,
        "Cybersecurity Specialist": 16000
    }
    cost = base_cost.get(role, 12000)
    return f"The estimated hiring cost for a {role} at SquareShift is approximately ${cost}."

cost_tool = Tool(
    name="CostEstimatorTool",
    func=estimate_cost,
    description="Estimates hiring cost for a given role at SquareShift"
)

# List of all tools
tools = [jd_tool, plan_tool, cost_tool]


In [11]:
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

# Step 1: Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Step 2: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)


In [21]:
# Example 1: Start a conversation
agent.run("We're hiring a Senior Software Engineer. Can you help me plan the hiring process?")



> Entering new AgentExecutor chain...
```json
{
    "action": "HiringPlanTool",
    "action_input": "Senior Software Engineer"
}
```
Observation: The hiring plan for a Senior Software Engineer at SquareShift includes: 1) Sourcing candidates through tech communities and platforms; 2) Screening resumes for alignment with our expertise in AI, ML, and cloud services; 3) Conducting technical interviews focusing on problem-solving and innovation; 4) Final interviews to assess cultural fit and commitment to our mission; 5) Rolling out offers with competitive compensation and growth opportunities.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The hiring plan for a Senior Software Engineer at SquareShift includes: 1) Sourcing candidates through tech communities and platforms; 2) Screening resumes for alignment with our expertise in AI, ML, and cloud services; 3) Conducting technical interviews focusing on problem-solving and innovation; 4) Final interviews to assess cul

'The hiring plan for a Senior Software Engineer at SquareShift includes: 1) Sourcing candidates through tech communities and platforms; 2) Screening resumes for alignment with our expertise in AI, ML, and cloud services; 3) Conducting technical interviews focusing on problem-solving and innovation; 4) Final interviews to assess cultural fit and commitment to our mission; 5) Rolling out offers with competitive compensation and growth opportunities.'

In [22]:
# Example 2: Ask about JD
agent.run("Can you generate a job description for this role?")



> Entering new AgentExecutor chain...
```json
{
    "action": "JobDescriptionGenerator",
    "action_input": "DevOps Engineer"
}
```
Observation: SquareShift is seeking a talented and experienced DevOps Engineer to join our dynamic team. As a DevOps Engineer at SquareShift, you will play a crucial role in driving digital transformation, data engineering, cybersecurity, and cloud adoption initiatives for our clients in the USA, Singapore, and India. 

Responsibilities:
- Collaborate with cross-functional teams to design, implement, and maintain scalable and secure infrastructure solutions
- Drive digital transformation efforts by automating processes, optimizing workflows, and implementing best practices
- Develop and maintain data engineering pipelines to ensure efficient data processing and analysis
- Implement cybersecurity measures to protect our clients' data and infrastructure
- Lead cloud adoption initiatives by designing and implementing cloud-based solutions
- Work closely wi

"SquareShift is seeking a talented and experienced DevOps Engineer to join our dynamic team. As a DevOps Engineer at SquareShift, you will play a crucial role in driving digital transformation, data engineering, cybersecurity, and cloud adoption initiatives for our clients in the USA, Singapore, and India. Responsibilities include collaborating with cross-functional teams, driving digital transformation efforts, developing data engineering pipelines, implementing cybersecurity measures, leading cloud adoption initiatives, working closely with clients, staying up-to-date on technologies, and providing technical guidance. Qualifications include a Bachelor's degree, 5+ years of experience, knowledge of cloud platforms, experience with automation tools, programming proficiency, communication skills, and ability to work in a fast-paced environment."

In [23]:
# Example 3: Cost?
agent.run("How much will this hire cost us?")



> Entering new AgentExecutor chain...
```json
{
    "action": "CostEstimatorTool",
    "action_input": "Senior Software Engineer"
}
```
Observation: The estimated hiring cost for a Senior Software Engineer at SquareShift is approximately $15000.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The estimated hiring cost for a Senior Software Engineer at SquareShift is approximately $15000."
}
```

> Finished chain.


'The estimated hiring cost for a Senior Software Engineer at SquareShift is approximately $15000.'